In [103]:
import pandas as pd
import numpy as np

In [104]:
pandemic = pd.read_csv("final/Pandemic_v1.csv")
prepandemic = pd.read_csv("final/prepandemic_v1.csv")
unemployment = pd.read_csv("final/Unemployment_v1.csv")

# Remove all Puerto Rico data
pandemic = pandemic.drop(pandemic[pandemic['stname'] == 'Puerto Rico'].index)

In [105]:
# Need state and county name
state = pandemic['stname']
county = pandemic['ctyname']

# The confirmed cases and death number is accumulated, used the last available date data
confirmed_cases = pandemic['confirmed_cases_20200418']
deaths = pandemic['deaths_20200418']

In [106]:
# Get days between first confirmed case to now
import datetime
def days(pandemic_row):
    start = datetime.datetime(2020,1,22)
    end = datetime.datetime(2020,4,18)
    days = 0
    while start <= end:
        title = 'confirmed_cases_' +  start.strftime("%Y%m%d")
        if pandemic_row[title] == 0:
            days += 1
        else:
            break

        start += datetime.timedelta(days=1)
    
    return days
case_days = pandemic.apply(lambda x: days(x), axis = 1) 

In [107]:
# Use year 2018(closest) population density
populationdensity=prepandemic['popdensity_2018']
populationdensity_min = populationdensity.min()
populationdensity_mean = populationdensity.mean()
populationdensity_stage = populationdensity_mean - populationdensity_min
def populationdensity_category(populationdensity):
    if populationdensity < populationdensity_mean - populationdensity_stage * 0.9: 
        return '1_Very Low' 
    elif populationdensity < populationdensity_mean - populationdensity_stage * 0.3: 
        return '2_Low'  
    elif populationdensity < populationdensity_mean + populationdensity_stage * 0.3: 
        return '3_Medium'   
    elif populationdensity < populationdensity_mean + populationdensity_stage: 
        return '4_High' 
    else:
        return 'Very_High'
populationdensity_category = prepandemic.apply(lambda x: populationdensity_category(x['popdensity_2018']), axis = 1)

In [108]:
# Get top 3 most populations ethnicity group in the county
def ethnic_group_top(hispanic, white, black, indian, asian, hawaii_na, top):
    ethnic_groups = [('Hispanic', hispanic), ('White', white), ('Black', black), ('Indian', indian), ('Asian', asian), ('Hawaii and N/A', hawaii_na)]
    ethnic_groups = sorted(ethnic_groups, key = lambda x: -x[1])
    return ethnic_groups[top - 1][0]

ethnic_group_top1 = prepandemic.apply(lambda x: ethnic_group_top(x['Hispanic2018'], x['nH_White_2018'], x['nH_Black_2018'], x['nH_Indian_Na_2018'], x['nH_Asian_2018'], x['nH_Hawaii_Na_2018'], 1), axis = 1)
ethnic_group_top2 = prepandemic.apply(lambda x: ethnic_group_top(x['Hispanic2018'], x['nH_White_2018'], x['nH_Black_2018'], x['nH_Indian_Na_2018'], x['nH_Asian_2018'], x['nH_Hawaii_Na_2018'], 2), axis = 1)
ethnic_group_top3 = prepandemic.apply(lambda x: ethnic_group_top(x['Hispanic2018'], x['nH_White_2018'], x['nH_Black_2018'], x['nH_Indian_Na_2018'], x['nH_Asian_2018'], x['nH_Hawaii_Na_2018'], 3), axis = 1)

In [109]:
# Age data has 18 columns, we group them to children, young adult, middle adult, senior
# column1:  age 0-4 prop.
# column2:  age 5-9 prop.
# column3:  age 10-14 prop.
# column4:  age 15-19 prop.
# column5:  age 20-24 prop.
# column6:  age 25-29 prop.
# column7:  age 30-34 prop.
# column8:  age 35-39 prop.
# column9:  age 40-44 prop.
# column10: age 45-49 prop.
# column11: age 50-54 prop.
# column12: age 55-59 prop.
# column13: age 60-64 prop.
# column14: age 65-69 prop.
# column15: age 70-74 prop.
# column16: age 75-79 prop.
# column17: age 80-84 prop.
# column18: age 85+ prop.
def age_group_top(age, top):
    children_poportion = age[0] + age[1] + age[2] + age[3]
    young_adult_poportion = age[4] + age[5] + age[6] + age[7] + age[8]
    middle_adult_poportion = age[9] + age[10] + age[11] + age[12]
    senior_poportion = age[13] + age[14] + age[15] + age[16] + age[17]
    age_group_list = [('Children', children_poportion), ('Young Adult', young_adult_poportion), ('Middle Adult', middle_adult_poportion), ('Senior', senior_poportion)]
    age_group_list = sorted(age_group_list, key = lambda x: -x[1])
    return age_group_list[top - 1][0]
age_groups_top1 = prepandemic.apply(lambda x: age_group_top([x['ageg1_2018'], x['ageg2_2018'], x['ageg3_2018'], x['ageg4_2018'], x['ageg5_2018'], x['ageg6_2018'], x['ageg7_2018'], x['ageg8_2018'], x['ageg9_2018'], x['ageg10_2018'], x['ageg11_2018'], x['ageg12_2018'], x['ageg13_2018'], x['ageg14_2018'], x['ageg15_2018'], x['ageg16_2018'], x['ageg17_2018'], x['ageg18_2018']], 1), axis = 1)
age_groups_top2 = prepandemic.apply(lambda x: age_group_top([x['ageg1_2018'], x['ageg2_2018'], x['ageg3_2018'], x['ageg4_2018'], x['ageg5_2018'], x['ageg6_2018'], x['ageg7_2018'], x['ageg8_2018'], x['ageg9_2018'], x['ageg10_2018'], x['ageg11_2018'], x['ageg12_2018'], x['ageg13_2018'], x['ageg14_2018'], x['ageg15_2018'], x['ageg16_2018'], x['ageg17_2018'], x['ageg18_2018']], 2), axis = 1)
age_groups_top3 = prepandemic.apply(lambda x: age_group_top([x['ageg1_2018'], x['ageg2_2018'], x['ageg3_2018'], x['ageg4_2018'], x['ageg5_2018'], x['ageg6_2018'], x['ageg7_2018'], x['ageg8_2018'], x['ageg9_2018'], x['ageg10_2018'], x['ageg11_2018'], x['ageg12_2018'], x['ageg13_2018'], x['ageg14_2018'], x['ageg15_2018'], x['ageg16_2018'], x['ageg17_2018'], x['ageg18_2018']], 3), axis = 1)
age_groups_top4 = prepandemic.apply(lambda x: age_group_top([x['ageg1_2018'], x['ageg2_2018'], x['ageg3_2018'], x['ageg4_2018'], x['ageg5_2018'], x['ageg6_2018'], x['ageg7_2018'], x['ageg8_2018'], x['ageg9_2018'], x['ageg10_2018'], x['ageg11_2018'], x['ageg12_2018'], x['ageg13_2018'], x['ageg14_2018'], x['ageg15_2018'], x['ageg16_2018'], x['ageg17_2018'], x['ageg18_2018']], 4), axis = 1)

In [110]:
# Get all education adata
education = pd.read_excel("final/Education_By_County.xls")
education = education.drop(education[pd.isna(education['2013 Rural-urban Continuum Code'])].index)

In [111]:
states_map = pd.read_csv("final/states.csv")
states_map

for index, row in states_map.iterrows():
    pc = len(pandemic[pandemic['stname'] == row[1]])
    ec = len(education[education['State'] == row[0]])
    if pc != ec:
        print(f"In education data set[{row[0]}]: {ec} rows, in pandemic dataset[{row[1]}]: {pc} rows")  

In education data set[VA]: 134 rows, in pandemic dataset[Virginia]: 133 rows
In education data set[PR]: 78 rows, in pandemic dataset[Puerto Rico]: 0 rows


In [112]:
va_pandemic = pandemic[pandemic['stname'] == 'Virginia']
va_education = education[education['State'] == 'VA']
for index, row in va_education.iterrows():
    area = row['Area name']
    if len(va_pandemic[va_pandemic['ctyname'] == area]) == 0:
        print(f"{area} is missing in va_pandemic")

Bedford city is missing in va_pandemic


In [113]:
education = education.drop(education[education['Area name'] == 'Bedford city'].index)

#education['Percent of adults completing some college or associate\'s degree, 2014-18']
# education['Percent of adults with a bachelor\'s degree or higher, 2014-18']

# Age data has 18 columns, we group them to children, young adult, middle adult, senior
# column1:  less than high school diploma poportion.
# column2:  high school diploma only poportion.
# column3:  some college or associate degress poportion.
# column4:  bachelor or higher education poportion.
def education_group_top(less_than_high_school, high_school, college, bachelor_above, top):
    education_group = [('Less_than_high_school', less_than_high_school), ('High_school', high_school), ('Some_college_or_associate', college), ('Bachelor_Or_Above', bachelor_above)]
    education_group = sorted(education_group, key = lambda x: -x[1])
    return education_group[top - 1][0]

education_group_top1 = education.apply(lambda x: education_group_top(x['Percent of adults with less than a high school diploma, 2014-18'], 
                                                                     x['Percent of adults with a high school diploma only, 2014-18'], 
                                                                     x['Percent of adults completing some college or associate\'s degree, 2014-18'], 
                                                                     x['Percent of adults with a bachelor\'s degree or higher, 2014-18'], 1), axis=1)
education_group_top2 = education.apply(lambda x: education_group_top(x['Percent of adults with less than a high school diploma, 2014-18'], 
                                                                     x['Percent of adults with a high school diploma only, 2014-18'], 
                                                                     x['Percent of adults completing some college or associate\'s degree, 2014-18'], 
                                                                     x['Percent of adults with a bachelor\'s degree or higher, 2014-18'], 2), axis=1)
education_group_top3 = education.apply(lambda x: education_group_top(x['Percent of adults with less than a high school diploma, 2014-18'], 
                                                                     x['Percent of adults with a high school diploma only, 2014-18'], 
                                                                     x['Percent of adults completing some college or associate\'s degree, 2014-18'], 
                                                                     x['Percent of adults with a bachelor\'s degree or higher, 2014-18'], 3), axis=1)
education_group_top4 = education.apply(lambda x: education_group_top(x['Percent of adults with less than a high school diploma, 2014-18'], 
                                                                     x['Percent of adults with a high school diploma only, 2014-18'], 
                                                                     x['Percent of adults completing some college or associate\'s degree, 2014-18'], 
                                                                     x['Percent of adults with a bachelor\'s degree or higher, 2014-18'], 4), axis=1)

In [114]:
populations = pd.read_excel("final/co-est2019-annres.xlsx")
populations['county'] = populations['Geographic Area'].apply(lambda x: x.split(',')[0][1:])
populations['state'] = populations['Geographic Area'].apply(lambda x: x.split(',')[1])
population = populations['Census']

In [115]:
result = pd.concat([state, county], axis=1)
                    
result['confirmed_cases'] = confirmed_cases
result['deaths'] = deaths
result['population'] = population
result['days'] = case_days
result['avearge_case'] = (100000.0) * confirmed_cases / case_days / population # Daily confirmed case per 100k people
result['populationdensity'] = populationdensity_category
result['toppest_ethnicity'] = ethnic_group_top1
result['second_ethnicity'] = ethnic_group_top2
result['third_ethnicity'] = ethnic_group_top3
result['toppest_age_group'] = age_groups_top1
result['second_age_group'] = age_groups_top2
result['third_age_group'] = age_groups_top3
result['fourth_age_group'] = age_groups_top4
result['top_education'] = education_group_top1
result['second_top_education'] = education_group_top2

In [116]:
# Target is 5 groups, very low, low, medium, high and very high
def target_group(rank, total):
    if (rank < total / 5.0):
        return "Very_Low"
    if (rank < total * 2.0 / 5.0):
        return "Low"
    if (rank < total * 3.0 / 5.0):
        return "Medium"
    if (rank < total * 4.0 / 5.0):
        return "High"
    return "Very_High"

total = len(result)

result['target_rank'] = result['avearge_case'].rank().apply(lambda r: target_group(r, total))

In [117]:
result

,stname,ctyname,confirmed_cases,deaths,population,days,avearge_case,populationdensity,toppest_ethnicity,second_ethnicity,third_ethnicity,toppest_age_group,second_age_group,third_age_group,fourth_age_group,top_education,second_top_education,target_rank
0,Alabama,Autauga County,25,2,54571,62,0.738901,2_Low,White,Black,Hispanic,Young Adult,Middle Adult,Children,Senior,High_school,Some_college_or_associate,Medium
1,Alabama,Baldwin County,109,2,182265,53,1.128359,2_Low,White,Black,Hispanic,Young Adult,Middle Adult,Children,Senior,Some_college_or_associate,Bachelor_Or_Above,High
2,Alabama,Barbour County,18,0,27457,72,0.910515,1_Very Low,Black,White,Hispanic,Young Adult,Middle Adult,Children,Senior,High_school,Less_than_high_school,High
3,Alabama,Bibb County,26,0,22915,68,1.668571,1_Very Low,White,Black,Hispanic,Young Adult,Middle Adult,Children,Senior,High_school,Some_college_or_associate,High
4,Alabama,Blount County,20,0,57322,63,0.553819,2_Low,White,Hispanic,Black,Young Adult,Middle Adult,Children,Senior,High_school,Some_college_or_associate,Medium
5,Alabama,Bullock County,9,0,10914,64,1.288483,1_Very Low,Black,White,Hispanic,Young Adult,Middle Adult,Children,Senior,High_school,Less_than_high_school,High
6,Alabama,Butler County,13,0,20947,63,0.985101,1_Very Low,White,Black,Hispanic,Young Adult,Middle Adult,Children,Senior,High_school,Some_college_or_associate,High
7,Alabama,Calhoun County,66,2,118572,56,0.993971,2_Low,White,Black,Hispanic,Young Adult,Middle Adult,Children,Senior,Some_college_or_associate,High_school,High
8,Alabama,Chambers County,240,11,34215,57,12.306083,2_Low,White,Black,Hispanic,Young Adult,Middle Adult,Children,Senior,High_school,Some_college_or_associate,Very_High
9,Alabama,Cherokee County,12,0,25989,63,0.732911,2_Low,White,Black,Hispanic,Middle Adult,Young Adult,Senior,Children,High_school,Some_college_or_associate,Medium


In [118]:
result.to_csv('final/model.csv')